# Kings County Data

We were given a dataset regarding housing prices in Kings County. Our task was to create a model that predicts selling price of a house.

In [ ]:
#import data file
import pandas as pd
import numpy as np
df = pd.read_csv("kc_house_data.csv")

## First Import and Clean Data

After importing the needed packages and exploring the data, it was clear that cleaning was needed. Specifically, there were placeholder symbols in some columns, some columns were the wrong data type and some had NaN values that needed to be replaced. After taking care of these things, there were features to engineer. In this case, I created an value for the age of the house, how much of the square feet was not in the basement, bathrooms per bedroom, and a boolean value for whether the house had a basement. 

In [ ]:
df["sqft_basement"].replace(to_replace="?", value="0.0", inplace=True) #fix placeholders
df["sqft_basement"] = df["sqft_basement"].astype(float, inplace=True) #reassign to float
df["basement"] = df["sqft_basement"].apply(lambda x: False if x == 0.0 else True) #create a boolean variable
df["yr_renovated"].fillna(value = 0) #deal with missing data
df["renovated"] = df["yr_renovated"].map(lambda x: x > 0, True) #boolean feature
df["age_of_house"] = 2019 - df["yr_built"] #engineer feature
df["upstairs_as_percent_of_house"] = (df["sqft_living"] - df["sqft_basement"]) / df["sqft_living"] #new feature
df["bath_per_bed"] = df["bathrooms"] / df["bedrooms"] #new feature

## Next Coding Was Needed

At this point I still needed a way to deal with zipcodes and I also needed a way to create 1 hot encoding for ordinal and categorical variables. For the zipcode data, I did outside research and grouped the zipcodes by high, midhigh, and mid average income for the area. Keep in mind, the mid is still a large average income as it is a wealthy area on the whole.  

In [11]:
#creating helpful lists
high_price_zipcodes = [98039, 98040, 98004, 98112]
midhigh_price_zipcodes = [98075, 98033, 98074, 98053, 98121, 98006, 98199]
mid_price_zipcodes = [98105, 98065, 98177, 98005, 98052, 98029, 98119, 98027 ,98072]
top_20_zipcodes = high_price_zipcodes + midhigh_price_zipcodes + mid_price_zipcodes

#creating a zipcode rank feature
zipcode_rank = []

for zipcode in df.zipcode:
    if zipcode in set(high_price_zipcodes):
        zipcode_rank.append("high")        
for zipcode in df.zipcode:
    if zipcode in set(midhigh_price_zipcodes):
        zipcode_rank.append("midhigh")       
for zipcode in df.zipcode:
    if zipcode in set(mid_price_zipcodes):
        zipcode_rank.append("mid")
for zipcode in df.zipcode:
    if zipcode not in set(top_20_zipcodes):
        zipcode_rank.append("other")

df["zipcode_rank"] = zipcode_rank

#recoding for final transform
zipcode_recode = []
for i in df["zipcode_rank"]:
    if i == "high":
        zipcode_recode.append(3)
    elif i == "midhigh":
        zipcode_recode.append(2)
    elif i == "mid":
        zipcode_recode.append(1)
    else:
        zipcode_recode.append(0)
df["zipcode_recode"] = zipcode_recode

I then created the 1 hot encoding columns for all ordinal and categorical variables.

In [12]:
#making dummy variables

bathroom_bins = [0, 2, 4, 8]
df["bath_bin"] = pd.cut(df["bathrooms"], bathroom_bins)
df.bath_bin.value_counts()

bedroom_bins = [0, 2, 3, 5, 33]
df["bed_bin"] = pd.cut(df["bedrooms"], bedroom_bins)
df.bed_bin.value_counts()

bed_dummy = pd.get_dummies(df.bed_bin, prefix="BED")
bath_dummy = pd.get_dummies(df.bath_bin, prefix="BATH")
df = pd.concat([df, bed_dummy, bath_dummy], axis=1)

condition_bin = [0, 1, 2, 3, 4, 5]
df["condition_bin"] = pd.cut(df["condition"], condition_bin)
condition_dummy = pd.get_dummies(df.condition_bin, prefix="COND")
df = pd.concat([df, condition_dummy], axis=1)

floor_bins = [0, 1, 2, 3, 4]
df["floor_bin"] = pd.cut(df["floors"], floor_bins)
floor_dummy = pd.get_dummies(df.floor_bin, prefix="FLOORS")
df = pd.concat([df, floor_dummy], axis=1)

zip_bin = [0, 1, 2, 3]
df["zip_bin"] = pd.cut(df["zipcode_recode"], zip_bin)
zip_dummy = pd.get_dummies(df.zip_bin, prefix="ZIP")
df = pd.concat([df, zip_dummy], axis=1)

view_bins = [0, 1, 2, 3, 4, 5]
df["view_bin"] = pd.cut(df["view"], view_bins)
view_dummy = pd.get_dummies(df.view_bin, prefix="VIEW")
df = pd.concat([df, view_dummy], axis=1)

grade_bins = [1, 3, 7, 11, 13]
df["grade_bin"] = pd.cut(df["grade"], grade_bins)
grade_dummy = pd.get_dummies(df.grade_bin, prefix="GRADE")
df = pd.concat([df, grade_dummy], axis=1)



## These Continuous Variables Aren't Normal

At this point we have all categorical and orginal variables recoded and now we need to perform transforms on the continuous data to help normalize it. To do this i used log transforms on all data except the age of the house. I was able to use min/max scaling for age of the house for a better fit for normality.

In [13]:
#variable transforms for normality

df["log_sqft_living15"] = np.log(df.sqft_living15)
df['log_sqft_lot15'] = np.log(df.sqft_lot15)
x_age = df.age_of_house
df["scale_age_of_house"] = (x_age - x_age.min()) / (x_age.max() - x_age.min())
df["log_sqft_living"] = np.log(df.sqft_living)
df["log_sqft_lot"] = np.log(df.sqft_lot)


## Drop the Columns Like They Are Hot

Our data frame is now much larger than it needs to be. First of all, we must drop the first column of all 1 hot encoding variables to prevent multicolinearity as well as the initial column the data was encoded from. Then we need to drop other colums that show high degree of multicolinearity.

In [17]:
#drop unneeded columns including the first colums of 1-hot encoded variables
df_clean = df.drop(columns = ['id', 'date', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15', 'age_of_house', 
       'upstairs_as_percent_of_house','zipcode_rank', 'zipcode_recode', 
       'bath_bin', 'bed_bin', 'BED_(0, 2]', 'BATH_(0, 2]', 'condition_bin', 
       'COND_(0, 1]', 'floor_bin', 'FLOORS_(0, 1]', 'zip_bin', 'view_bin', 
       'VIEW_(0, 1]', 'grade_bin', 'GRADE_(1, 3]'])

## Remove Outliers for Better Modeling

Now we need to remove the outliers, create and verify the model. First I removed all houses over 7 million dollars. Then the appropriate libraries for modeling were imported and two models were created. The first used all of the remaining columns of the data frame. The second removed a few variables that were on the boarder of our cuttoff for multicolinearity. 

In [18]:
#remove price outliers
df_clean = df_clean[df_clean.price < 7000000]

In [28]:
#prepare for modeling 1
features = df_clean.drop(columns = "price")
target = df_clean["price"]
X1 = features
y1 = target

#1st Model
import statsmodels.api as sm
X_int_sm1 = sm.add_constant(X1)
model1 = sm.OLS(y1.astype(float), X_int_sm1.astype(float)).fit()
model1.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.593
Model:                            OLS   Adj. R-squared:                  0.592
Method:                 Least Squares   F-statistic:                     1120.
Date:                Fri, 31 May 2019   Prob (F-statistic):               0.00
Time:                        18:14:13   Log-Likelihood:            -2.9732e+05
No. Observations:               21595   AIC:                         5.947e+05
Df Residuals:                   21566   BIC:                         5.949e+05
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
basement           -3.681e+06   2.41e+05    -15.279      0.000   -4.15e+06   -3.21e+06
renovated           5.413e+04   9124.851      5.932      0.000    3.62e+04     7.2e+04
bath_per_bed         1.02e+05   1.28e+04      7.946      0.000    7.69e+04    1.27e+05
BED_(2, 3]         -6.627e+04   6073.926    -10.911      0.000   -7.82e+04   -5.44e+04
BED_(3, 5]         -7.297e+04   8418.807     -8.667      0.000   -8.95e+04   -5.65e+04
BED_(5, 33]        -1.167e+05   1.66e+04     -7.043      0.000   -1.49e+05   -8.42e+04
BATH_(2, 4]        -2.841e+04   5566.929     -5.103      0.000   -3.93e+04   -1.75e+04
BATH_(4, 8]         4.384e+05   1.74e+04     25.238      0.000    4.04e+05    4.72e+05
COND_(1, 2]         6.617e+04   4.65e+04      1.424      0.154   -2.49e+04    1.57e+05
COND_(2, 3]         8.475e+04   4.31e+04      1.965      0.049     225.486    1.69e+05
COND_(3, 4]         9.691e+04   4.31e+04      2.247      0.025    1.24e+04    1.81e+05
COND_(4, 5]         1.435e+05   4.34e+04      3.307      0.001    5.85e+04    2.29e+05
FLOORS_(1, 2]       2.148e+04   3946.692      5.444      0.000    1.37e+04    2.92e+04
FLOORS_(2, 3]        1.43e+05   9660.350     14.798      0.000    1.24e+05    1.62e+05
FLOORS_(3, 4]       2.053e+05   8.76e+04      2.344      0.019    3.36e+04    3.77e+05
ZIP_(0, 1]          -245.5383   4815.518     -0.051      0.959   -9684.309    9193.233
ZIP_(1, 2]         -4335.7065   5037.513     -0.861      0.389   -1.42e+04    5538.192
ZIP_(2, 3]         -1.246e+04   7850.836     -1.587      0.112   -2.78e+04    2927.642
VIEW_(1, 2]         6.817e+04   7827.171      8.710      0.000    5.28e+04    8.35e+04
VIEW_(2, 3]         1.525e+05   1.06e+04     14.337      0.000    1.32e+05    1.73e+05
VIEW_(3, 4]         5.137e+05   1.35e+04     38.104      0.000    4.87e+05     5.4e+05
VIEW_(4, 5]           1.7e-08   7.93e-09      2.143      0.032    1.45e-09    3.25e-08
GRADE_(3, 7]        -8.45e+04   2.31e+05     -0.366      0.715   -5.37e+05    3.68e+05
GRADE_(7, 11]       2.106e+04   2.31e+05      0.091      0.927   -4.32e+05    4.74e+05
GRADE_(11, 13]       1.01e+06   2.32e+05      4.346      0.000    5.55e+05    1.47e+06
log_sqft_living15   2.184e+05   7896.729     27.656      0.000    2.03e+05    2.34e+05
log_sqft_lot15     -1.636e+04   4985.585     -3.282      0.001   -2.61e+04   -6591.247
scale_age_of_house  3.068e+05   8724.034     35.169      0.000     2.9e+05    3.24e+05
log_sqft_living     3.492e+05   7932.288     44.017      0.000    3.34e+05    3.65e+05
log_sqft_lot       -1.349e+04   4525.209     -2.981      0.003   -2.24e+04   -4620.277
==============================================================================
Omnibus:                    12545.791   Durbin-Watson:                   1.978
Prob(Omnibus):                  0.000   Jarque-Bera (J

In [25]:
#prepare for modeling 2
features2 = features.drop(columns = ["basement", "VIEW_(4, 5]", "log_sqft_living15", "log_sqft_lot15"])
target2 = df_clean["price"]
X2 = features2
y2 = target2

#1st Model
import statsmodels.api as sm
X_int_sm2 = sm.add_constant(X2)
model2 = sm.OLS(y2.astype(float), X_int_sm2.astype(float)).fit()
model2.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.578
Model:                            OLS   Adj. R-squared:                  0.578
Method:                 Least Squares   F-statistic:                     1137.
Date:                Fri, 31 May 2019   Prob (F-statistic):               0.00
Time:                        17:59:43   Log-Likelihood:            -2.9769e+05
No. Observations:               21595   AIC:                         5.954e+05
Df Residuals:                   21568   BIC:                         5.957e+05
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const              -2.758e+06   2.43e+05    -11.365      0.000   -3.23e+06   -2.28e+06
renovated           4.159e+04   9268.135      4.487      0.000    2.34e+04    5.98e+04
bath_per_bed        8.903e+04   1.31e+04      6.821      0.000    6.34e+04    1.15e+05
BED_(2, 3]         -7.663e+04   6168.685    -12.422      0.000   -8.87e+04   -6.45e+04
BED_(3, 5]          -8.46e+04   8554.717     -9.889      0.000   -1.01e+05   -6.78e+04
BED_(5, 33]        -1.533e+05   1.68e+04     -9.127      0.000   -1.86e+05    -1.2e+05
BATH_(2, 4]        -1.816e+04   5651.980     -3.213      0.001   -2.92e+04   -7079.156
BATH_(4, 8]         4.426e+05   1.77e+04     25.041      0.000    4.08e+05    4.77e+05
COND_(1, 2]         2.366e+04   4.72e+04      0.501      0.616   -6.89e+04    1.16e+05
COND_(2, 3]         4.301e+04   4.38e+04      0.981      0.327   -4.29e+04    1.29e+05
COND_(3, 4]         5.282e+04   4.38e+04      1.205      0.228   -3.31e+04    1.39e+05
COND_(4, 5]         9.306e+04   4.41e+04      2.110      0.035    6628.452    1.79e+05
FLOORS_(1, 2]       2.266e+04   4012.936      5.646      0.000    1.48e+04    3.05e+04
FLOORS_(2, 3]       1.211e+05   9781.771     12.378      0.000    1.02e+05     1.4e+05
FLOORS_(3, 4]       1.769e+05   8.91e+04      1.986      0.047    2304.291    3.52e+05
ZIP_(0, 1]          1779.6280   4899.351      0.363      0.716   -7823.462    1.14e+04
ZIP_(1, 2]         -4330.7444   5125.707     -0.845      0.398   -1.44e+04    5716.020
ZIP_(2, 3]         -1.097e+04   7988.046     -1.373      0.170   -2.66e+04    4686.064
VIEW_(1, 2]         8.486e+04   7940.742     10.687      0.000    6.93e+04       1e+05
VIEW_(2, 3]          1.75e+05   1.08e+04     16.215      0.000    1.54e+05    1.96e+05
VIEW_(3, 4]         5.379e+05   1.37e+04     39.313      0.000    5.11e+05    5.65e+05
GRADE_(3, 7]       -1.479e+05   2.35e+05     -0.629      0.529   -6.09e+05    3.13e+05
GRADE_(7, 11]      -1.536e+04   2.35e+05     -0.065      0.948   -4.76e+05    4.46e+05
GRADE_(11, 13]      9.957e+05   2.37e+05      4.209      0.000    5.32e+05    1.46e+06
scale_age_of_house  2.956e+05   8833.694     33.458      0.000    2.78e+05    3.13e+05
log_sqft_living      4.42e+05   7312.312     60.452      0.000    4.28e+05    4.56e+05
log_sqft_lot       -1.517e+04   2091.430     -7.255      0.000   -1.93e+04   -1.11e+04
==============================================================================
Omnibus:                    12233.677   Durbin-Watson:                   1.977
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           290691.836
Skew:                           2.256   Prob(JB):                         0.00
Kurtosis:                      20.398   Cond. No.                     3.51e+03
==============================================================================

## Data Modled with Scikit Learn and MSE and MSRE Calculated

I also created models with each of the two data frames with scikit learn to use the cross validation option. Then the mean squared error and the mean square root error were calulated for each model.

In [29]:
#Model 1.2
from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size = 0.20)
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train1, y_train1)
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)
y_hat_train1 = linreg.predict(X_train1)
y_hat_test1 = linreg.predict(X_test1)
train_residuals1 = y_hat_train1 - y_train1
test_residuals1 = y_hat_test1 - y_test1
from sklearn.metrics import mean_squared_error
train_mse1 = mean_squared_error(y_train1, y_hat_train1)
test_mse1 = mean_squared_error(y_test1, y_hat_test1)
train_MSRE1 = np.sqrt(train_mse1)
test_MSRE1 = np.sqrt(test_mse1)
print('Train Mean Squarred Error1:', train_mse1)
print('Test Mean Squarred Error1:', test_mse1)
print('Train Mean Square Root Error1', train_MSRE1)
print('Test Mean Square Root Error1', test_MSRE1)

Train Mean Squarred Error1: 54516557076.60007
Test Mean Squarred Error1: 48294342892.66986
Train Mean Square Root Error1 233487.8092676362
Test Mean Square Root Error1 219759.7390166585


In [30]:
#Model 2.2

from sklearn.model_selection import train_test_split
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size = 0.20)
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train2, y_train2)
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)
y_hat_train2 = linreg.predict(X_train2)
y_hat_test2 = linreg.predict(X_test2)
train_residuals2 = y_hat_train2 - y_train2
test_residuals2 = y_hat_test2 - y_test2
from sklearn.metrics import mean_squared_error
train_mse2 = mean_squared_error(y_train2, y_hat_train2)
test_mse2 = mean_squared_error(y_test2, y_hat_test2)
train_MSRE2 = np.sqrt(train_mse2)
test_MSRE2 = np.sqrt(test_mse2)
print('Train Mean Squarred Error2:', train_mse2)
print('Test Mean Squarred Error2:', test_mse2)
print('Train Mean Square Root Error2', train_MSRE2)
print('Test Mean Square Root Error2', test_MSRE2)

Train Mean Squarred Error2: 54191761485.7682
Test Mean Squarred Error2: 58975701049.680244
Train Mean Square Root Error2 232791.2401396758
Test Mean Square Root Error2 242849.13228109392


## Lastly V-Fold Cross Validation

Here v-fold cross validation was performed and then the mean squared error was calculated for both models. 

In [32]:
#Cross_Val Model 1.3
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

cv_5_results1 = cross_val_score(linreg, X1, y1, cv=5, scoring="neg_mean_squared_error")
np.sqrt(-1*(cv_5_results1))

array([2.62930429e+15, 2.30137459e+05, 2.19539322e+05, 2.23039033e+05,
       2.51333551e+05])

In [33]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

cv_5_results2 = cross_val_score(linreg, X2, y2, cv=5, scoring="neg_mean_squared_error")
np.sqrt(-1*(cv_5_results2))

array([1.53738082e+16, 2.35357359e+05, 2.25014163e+05, 2.28820582e+05,
       2.49074069e+05])